In [3]:
### 데이터 로딩
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
file = '../data/공정데이터 추가자료.csv'
data = pd.read_csv(file)

In [4]:
### 데이터 전처리
# 불필요 컬럼(Unnamed: 12 -> 데이터 값이 없거나 0) 제거
data = data.drop(columns=['Unnamed: 12'])


# 'time' 컬럼을 datetime 형식으로 변환
data['time'] = pd.to_datetime(data['time']).dt.tz_localize(None)


# 10월 1일을 기준으로 데이터 분할
oct_1 = pd.Timestamp('2023-10-01')


# 10월 이전 데이터
before_oct_data = data[data['time'] < oct_1]


# 10월 이후 데이터
after_oct_data = data[data['time'] >= oct_1]


# 불규칙적으로 측정된 'time' 컬럼 제거
set_data = before_oct_data.drop(columns=['time'])  # 피쳐공학적 접근에 따른 데이터 => set_data


# 모든 설정값이 동일한 s_temp_sv와 c_temp_sv컬럼 제거
set_data = set_data.drop(columns=['s_temp_sv', 'c_temp_sv'])


# 2차 미팅 결과 E_scr_sv가 8이 아닌 다른 값으로 셋팅된 것은 다른 제품에 대한 셋팅값으로 판단, E_scr_sv 값이 8인 행만 남기기
set_data = set_data[set_data['E_scr_sv'] == 8]


# E_scr_sv 값이 모두 8이므로 학습에 변화량을 제공하지 않아 삭제
set_data = set_data.drop(columns=['E_scr_sv'])


# E_scr_sv 컬럼 삭제 후 확인 시 n_temp_sv 컬럼이 모두 같은 값으로 통일되었기에 해당 컬럼을 삭제
set_data = set_data.drop(columns=['n_temp_sv'])


# E_scr_pv 컬럼의 고유값 확인시 8과 0으로 나옴 -> E_scr_pv 값이 8인 경우와 0인 경우로 데이터 나누기
data_8 = set_data[set_data['E_scr_pv'] == 8]
data_0 = set_data[set_data['E_scr_pv'] == 0]


## E_scr_pv 값이 0인 경우 k_rpm_pv가 모두 0이고, scale_pv값이 0 또는 600이상 값으로만 나옴
# scale_pv 값이 600 이상인 행은 기준이 되는 3과 너무 큰 차이가 나므로 측정 오류로 판단, 모두 삭제
final_data = set_data[set_data['scale_pv'] < 600]


# scale_pv 값이 4 이상인 행 또한 측정 오류로 판단, 모두 삭제
final_data = final_data[final_data['scale_pv'] < 4]


# c_temp_pv 값이 혼자 65.1인 날 E_scr_pv, k_rpm_pv, scale_pv값이 모두 0이므로 삭제, 남은 c_temp_pv 값이 모두 68 이상이므로 c_temp_pv 값이 68 미만인 행 삭제
final_data = final_data[final_data['c_temp_pv'] >= 68]


# 실제 칼날 회전수로 파악하는 방식 결정, k_rpm_sv 컬럼 제거
final_data = final_data.drop(columns=['k_rpm_sv'])


# E_scr_pv값이 0인 모든 케이스가  k_rpm_sv값이 50미만에 포함, 해당 값들은 scale_pv도 대부분 0 이므로 제외 -> 남은 E_scr_pv 모두 8이라 의미가 없으므로 E_scr_pv 컬럼 제거
final_data = final_data.drop(columns=['E_scr_pv'])


# k_rpm_pv값이 50미만 값들은 타겟이 3g인 값들에 대한 k_rpm_pv값 차이가 커 오류로 판단, scale_pv 값이 50 미만인 행 삭제
final_data = final_data[final_data['k_rpm_pv'] >= 50]


# scale_pv값이 0 인 경우 미측정, 나머지 0 초과 2 이하의 경우 측정오류로 판단, 제거
final_data = final_data[(final_data['scale_pv'] == 0) | (final_data['scale_pv'] > 2)]


# CSV 파일로 저장
output_file = '../data_final/Train_data.csv'
final_data.to_csv(output_file, index=False)

### 최종적으로 미측정값 scale_pv = 0에 대한 KNN 데이터 증강 실시 